In [1]:
#imports packages
import json
import collections
import more_itertools
import seaborn
import pandas as pd
import random
import subprocess
import pathlib
import numpy as np
import datetime
import tqdm
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import mmap
from tqdm import tqdm
import openpyxl
import more_itertools

In [2]:
#data -- import/upload xlsx spreadsheet
#data = pd.read_excel('/pscratch/sd/m/masare/sacct/incident_sla_AY23_complete.xlsx') #full data
data = pd.read_excel('/pscratch/sd/m/masare/sacct/incident_sla.xlsx') #small size

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2816 entries, 0 to 2815
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Number                            2816 non-null   object        
 1   Resource                          2139 non-null   object        
 2   Created                           2816 non-null   datetime64[ns]
 3   User ID                           2651 non-null   object        
 4   Assigned group                    2816 non-null   object        
 5   User ID.1                         2816 non-null   object        
 6   Staff Owner                       2699 non-null   object        
 7   User ID.2                         2699 non-null   object        
 8   Business elapsed time             2816 non-null   int64         
 9   Actual elapsed percentage         2816 non-null   float64       
 10  Title                             2816 non-null 

In [4]:
data.head(2)

,Number,Resource,Created,User ID,Assigned group,User ID.1,Staff Owner,User ID.2,Business elapsed time,Actual elapsed percentage,...,Has breached,Channel,Additional comments,Close code,Close notes,Comments and Work notes,Description,Staff work notes (NERSC private),ACS,Share with NERSC Projects
0,INC0197059,Cori,2023-01-18 04:39:28,lgerhard,DAS,stotor,"Gerhardt, Lisa (lgerhard)",lgerhard,274920,239.75,...,True,User Web Interface,2023-02-21 02:00:05 - System (Additional comme...,Not Solved (No User Response),No user response,2023-02-21 02:00:05 - System (Additional comme...,NaN,2023-01-30 10:46:55 - Kirill Lozinskiy (Staff ...,NaN,NaN
1,INC0197077,Iris,2023-01-18 07:34:40,bagwell,Account Support,pederson,"Bagwell, Clayton (bagwell)",bagwell,7675,2.12,...,False,Email,2023-02-09 02:00:05 - System (Additional comme...,Not Solved (No User Response),No user response,2023-02-09 02:00:05 - System (Additional comme...,NaN,NaN,NaN,NaN


In [5]:
data.columns

Index(['Number', 'Resource', 'Created', 'User ID', 'Assigned group',
       'User ID.1', 'Staff Owner', 'User ID.2', 'Business elapsed time',
       'Actual elapsed percentage', 'Title', 'User Category', 'Category',
       'Closure category', 'Has breached', 'Channel', 'Additional comments',
       'Close code', 'Close notes', 'Comments and Work notes', 'Description',
       'Staff work notes (NERSC private)', 'ACS', 'Share with NERSC Projects'],
      dtype='object')

In [6]:
#Rename Colum 'Number' to 'ID' into the dataframe
data = data.rename(columns={"Number":"ID"})
data.columns # RECHECK COLUMNS

Index(['ID', 'Resource', 'Created', 'User ID', 'Assigned group', 'User ID.1',
       'Staff Owner', 'User ID.2', 'Business elapsed time',
       'Actual elapsed percentage', 'Title', 'User Category', 'Category',
       'Closure category', 'Has breached', 'Channel', 'Additional comments',
       'Close code', 'Close notes', 'Comments and Work notes', 'Description',
       'Staff work notes (NERSC private)', 'ACS', 'Share with NERSC Projects'],
      dtype='object')

In [7]:
type(data)
data.dtypes

ID                                          object
Resource                                    object
Created                             datetime64[ns]
User ID                                     object
Assigned group                              object
User ID.1                                   object
Staff Owner                                 object
User ID.2                                   object
Business elapsed time                        int64
Actual elapsed percentage                  float64
Title                                       object
User Category                               object
Category                                    object
Closure category                           float64
Has breached                                  bool
Channel                                     object
Additional comments                         object
Close code                                  object
Close notes                                 object
Comments and Work notes        

In [8]:
data['ID']=data['ID'].astype(str) #ensuring ID column is of string type
data['ID']=data['ID'].str[3:] #removing first three characters
data['ID']=pd.to_numeric(data['ID'],errors='coerce') #converting ID column into numberic
data.head()

,ID,Resource,Created,User ID,Assigned group,User ID.1,Staff Owner,User ID.2,Business elapsed time,Actual elapsed percentage,...,Has breached,Channel,Additional comments,Close code,Close notes,Comments and Work notes,Description,Staff work notes (NERSC private),ACS,Share with NERSC Projects
0,197059,Cori,2023-01-18 04:39:28,lgerhard,DAS,stotor,"Gerhardt, Lisa (lgerhard)",lgerhard,274920,239.75,...,True,User Web Interface,2023-02-21 02:00:05 - System (Additional comme...,Not Solved (No User Response),No user response,2023-02-21 02:00:05 - System (Additional comme...,NaN,2023-01-30 10:46:55 - Kirill Lozinskiy (Staff ...,NaN,NaN
1,197077,Iris,2023-01-18 07:34:40,bagwell,Account Support,pederson,"Bagwell, Clayton (bagwell)",bagwell,7675,2.12,...,False,Email,2023-02-09 02:00:05 - System (Additional comme...,Not Solved (No User Response),No user response,2023-02-09 02:00:05 - System (Additional comme...,NaN,NaN,NaN,NaN
2,197078,Iris,2023-01-18 07:34:43,bagwell,Account Support,akara,"Bagwell, Clayton (bagwell)",bagwell,6720,1.90,...,False,Email,2023-02-09 02:00:06 - System (Additional comme...,Not Solved (No User Response),No user response,2023-02-09 02:00:06 - System (Additional comme...,NaN,NaN,NaN,NaN
3,197079,Iris,2023-01-18 07:34:45,mdscogg,Account Support,fpaesani,"Scoggins, Michelle (mdscogg)",mdscogg,2563,0.94,...,False,Email,2023-02-09 02:00:06 - System (Additional comme...,Not Solved (No User Response),No user response,2023-02-09 02:00:06 - System (Additional comme...,NaN,NaN,NaN,NaN
4,197080,Iris,2023-01-18 07:38:44,bagwell,Account Support,ching,"Bagwell, Clayton (bagwell)",bagwell,9955,2.59,...,False,Email,2023-02-09 02:00:06 - System (Additional comme...,Not Solved (No User Response),No user response,2023-02-09 02:00:06 - System (Additional comme...,NaN,NaN,NaN,NaN


In [9]:
#selecting desired columns
columns_to_keep = ['ID',
                 'Created',  
                'Title',
                'Additional comments',
                'Close notes',
                'Comments and Work notes',
                'Staff work notes (NERSC private)']
data = data.loc[:, columns_to_keep].copy()

In [10]:
#removing spaces from column names
data.columns = data.columns.str.replace(' ', '')

In [11]:
data.loc[:, 'Created']= pd.to_datetime(data['Created'], errors = 'coerce')

In [12]:
print(data.dtypes)

ID                                       int64
Created                         datetime64[ns]
Title                                   object
Additionalcomments                      object
Closenotes                              object
CommentsandWorknotes                    object
Staffworknotes(NERSCprivate)            object
dtype: object


In [13]:
data.columns

Index(['ID', 'Created', 'Title', 'Additionalcomments', 'Closenotes',
       'CommentsandWorknotes', 'Staffworknotes(NERSCprivate)'],
      dtype='object')

In [14]:
#convert columns from object to str. I have to use ".loc" to avoid the copy warning
object_columns =[ 'Title', 
                 'Additionalcomments', 
                 'Closenotes',
                 'CommentsandWorknotes', 
                 'Staffworknotes(NERSCprivate)']
for col in object_columns:
    data.loc[:, col] = data.loc[:, col].astype(str)

In [15]:
data.head(2)

,ID,Created,Title,Additionalcomments,Closenotes,CommentsandWorknotes,Staffworknotes(NERSCprivate)
0,197059,2023-01-18 04:39:28,Scratch purge extension,2023-02-21 02:00:05 - System (Additional comme...,No user response,2023-02-21 02:00:05 - System (Additional comme...,2023-01-30 10:46:55 - Kirill Lozinskiy (Staff ...
1,197077,2023-01-18 07:34:40,RE: Your NERSC account in the new allocation year,2023-02-09 02:00:05 - System (Additional comme...,No user response,2023-02-09 02:00:05 - System (Additional comme...,nan


In [16]:
data['Additionalcomments'][0]

"2023-02-21 02:00:05 - System (Additional comments)\nIncident automatically closed after 7 days in the Resolved state.\n\n2023-02-13 12:26:09 - System (Additional comments)\nIncident automatically resolved after 14 days in waiting for user reply.\n\n2023-01-18 12:25:50 - Lisa Gerhardt (Additional comments)\nHi Sam,\nOnce thI can extend your Cori scratch purging exemption until March 31, 2023. At this time, we don't have a good way to enforce a purge exemption on Perlmutter scratch (though we're also currently not purging), so I'd advise you to prioritize backing up that data.\n\nBye,\nLisa\n\n2023-01-18 04:39:29 - Samuel Totorica (Additional comments)\nHello,\n\nWould it be possible to reset the purging timer for my Cori and Perlmutter scratch folders?  I'm actively using the data in there for a couple of papers simultaneously right now.  In that time I will make sure to learn to use tape storage, but the extension would be a great help because I'm currently quite overwhelmed and it se

In [17]:
#clean 
   #isolate import columns, drop unimportant columns
   #remove time stamps (can look at my code and use that) 


text_cols = [ 'Created', 
             'Title',
            'Additionalcomments',
             'Closenotes',
            'CommentsandWorknotes',
           'Staffworknotes(NERSCprivate)'
            ]

data = data[['ID'] + text_cols].copy()
data.fillna('', inplace=True)
data = data.astype(str)

## EP -- Let's use this cell to name the columns we want our data frame. This is a little different than what I did in
# 'ticket_text_mine.py`

In [18]:
data['Additionalcomments'][0]

"2023-02-21 02:00:05 - System (Additional comments)\nIncident automatically closed after 7 days in the Resolved state.\n\n2023-02-13 12:26:09 - System (Additional comments)\nIncident automatically resolved after 14 days in waiting for user reply.\n\n2023-01-18 12:25:50 - Lisa Gerhardt (Additional comments)\nHi Sam,\nOnce thI can extend your Cori scratch purging exemption until March 31, 2023. At this time, we don't have a good way to enforce a purge exemption on Perlmutter scratch (though we're also currently not purging), so I'd advise you to prioritize backing up that data.\n\nBye,\nLisa\n\n2023-01-18 04:39:29 - Samuel Totorica (Additional comments)\nHello,\n\nWould it be possible to reset the purging timer for my Cori and Perlmutter scratch folders?  I'm actively using the data in there for a couple of papers simultaneously right now.  In that time I will make sure to learn to use tape storage, but the extension would be a great help because I'm currently quite overwhelmed and it se

In [19]:
data.head(2)

,ID,Created,Title,Additionalcomments,Closenotes,CommentsandWorknotes,Staffworknotes(NERSCprivate)
0,197059,2023-01-18 04:39:28,Scratch purge extension,2023-02-21 02:00:05 - System (Additional comme...,No user response,2023-02-21 02:00:05 - System (Additional comme...,2023-01-30 10:46:55 - Kirill Lozinskiy (Staff ...
1,197077,2023-01-18 07:34:40,RE: Your NERSC account in the new allocation year,2023-02-09 02:00:05 - System (Additional comme...,No user response,2023-02-09 02:00:05 - System (Additional comme...,nan


In [20]:
data[text_cols[5]]

0       2023-01-30 10:46:55 - Kirill Lozinskiy (Staff ...
1                                                     nan
2                                                     nan
3                                                     nan
4                                                     nan
                              ...                        
2811    2023-07-05 13:23:03 - Yun (Helen) He (Staff wo...
2812    2023-07-05 13:12:44 - Christopher Samuel (Staf...
2813                                                  nan
2814                                                  nan
2815                                                  nan
Name: Staffworknotes(NERSCprivate), Length: 2816, dtype: object

In [21]:
def process_comment_text(text):
    
    # Removed System Messages
    text = re.sub(r'No user response \d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\s- System \(Additional comments\)','',text)
    text = re.sub(r'\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\s-.*?\(Additional\scomments\)','',text)
    text = re.sub(r'\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\s-.{1,60}?\(Staff work notes \(NERSC private\)\)','',text)
    
    text = re.sub(r'On\s\w{3},\s\w{3}\s\d+,\s\d{4}\sat\s\d+:\d{2}\s\w{2}','',text)
    text = re.sub(r'Incident automatically closed after \d+? days in the Resolved state.','',text)
    text = re.sub(r'Incident automatically resolved after \d+? days in waiting for user reply','',text)
                    #Incident automatically resolved after 14 days in waiting for user reply.
    
    # Remove HTML links
    text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', text)
    
    #remove phonenumber
    text = re.sub(r'(\+?\d{1,2}[-.\s]?)?(\(?\d{3}\)?[-.\s]?)?\d{3}[-.\s]?\d{4}', '', text)
    
    #remove emails
    text = re.sub(r'[\w]+@[\w]+.\w{3}','',text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove specific words like 'hi', 'hello'
    unwanted_words = ['hi', 'hello','email','organization','laboratory','bye','mr','thanks','day']
    pattern = r'\b(?:{})\b'.format('|'.join(unwanted_words))
    text = re.sub(pattern, '', text)
    

    return text

In [33]:
%%time
for col in text_cols: 
    if col not in data.columns: 
        raise ValueError(f"Column '{col}' not found in DataFrame") 


data[text_cols] = data[text_cols].fillna('').astype(str)
# Concatenate the text columns
print(data.head())
data['text_data'] = data[text_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
print(data.head())
# Apply the cleaning function
data_before = pd.DataFrame()
data_before['text_data'] = data['text_data']
data['text_data'] = data['text_data'].apply(process_comment_text)

       ID              Created  \
0  197059  2023-01-18 04:39:28   
1  197077  2023-01-18 07:34:40   
2  197078  2023-01-18 07:34:43   
3  197079  2023-01-18 07:34:45   
4  197080  2023-01-18 07:38:44   

                                               Title  \
0                            Scratch purge extension   
1  RE: Your NERSC account in the new allocation year   
2  Re: Your NERSC account in the new allocation year   
3  Re: Your NERSC account in the new allocation year   
4  RE: Your NERSC account in the new allocation year   

                                  Additionalcomments        Closenotes  \
0  2023-02-21 02:00:05 - System (Additional comme...  No user response   
1  2023-02-09 02:00:05 - System (Additional comme...  No user response   
2  2023-02-09 02:00:06 - System (Additional comme...  No user response   
3  2023-02-09 02:00:06 - System (Additional comme...  No user response   
4  2023-02-09 02:00:06 - System (Additional comme...  No user response   

             

In [23]:
data_cleaned = data[['ID','Created','text_data']] 

In [24]:
data_before['text_data'].iloc[0]

"2023-01-18 04:39:28 Scratch purge extension 2023-02-21 02:00:05 - System (Additional comments)\nIncident automatically closed after 7 days in the Resolved state.\n\n2023-02-13 12:26:09 - System (Additional comments)\nIncident automatically resolved after 14 days in waiting for user reply.\n\n2023-01-18 12:25:50 - Lisa Gerhardt (Additional comments)\nHi Sam,\nOnce thI can extend your Cori scratch purging exemption until March 31, 2023. At this time, we don't have a good way to enforce a purge exemption on Perlmutter scratch (though we're also currently not purging), so I'd advise you to prioritize backing up that data.\n\nBye,\nLisa\n\n2023-01-18 04:39:29 - Samuel Totorica (Additional comments)\nHello,\n\nWould it be possible to reset the purging timer for my Cori and Perlmutter scratch folders?  I'm actively using the data in there for a couple of papers simultaneously right now.  In that time I will make sure to learn to use tape storage, but the extension would be a great help becau

In [25]:
process_comment_text(data_before['text_data'].iloc[0])

"2023-01-18 04:39:28 scratch purge extension \n\n\n\n.\n\n\n sam,\nonce thi can extend your cori scratch purging exemption until march 31, 2023. at this time, we don't have a good way to enforce a purge exemption on perlmutter scratch (though we're also currently not purging), so i'd advise you to prioritize backing up that data.\n\n,\nlisa\n\n\n,\n\nwould it be possible to reset the purging timer for my cori and perlmutter scratch folders?  i'm actively using the data in there for a couple of papers simultaneously right now.  in that time i will make sure to learn to use tape storage, but the extension would be a great help because i'm currently quite overwhelmed and it seems tape may be complicated as some of the simulations have a number of files that exceed the the htar limit.  i would really appreciate this!\n\nthe folders are:\n\n/global/cscratch1/sd/stotor\n/pscratch/sd/s/stotor\n\nthank you,\nsam\n\n \n\n\n\n.\n\n\n lisa, ! i added the path to the exemption list for purging. \n

In [26]:
data_cleaned['text_data'].iloc[0]

"2023-01-18 04:39:28 Scratch purge extension 2023-02-21 02:00:05 - System (Additional comments)\nIncident automatically closed after 7 days in the Resolved state.\n\n2023-02-13 12:26:09 - System (Additional comments)\nIncident automatically resolved after 14 days in waiting for user reply.\n\n2023-01-18 12:25:50 - Lisa Gerhardt (Additional comments)\nHi Sam,\nOnce thI can extend your Cori scratch purging exemption until March 31, 2023. At this time, we don't have a good way to enforce a purge exemption on Perlmutter scratch (though we're also currently not purging), so I'd advise you to prioritize backing up that data.\n\nBye,\nLisa\n\n2023-01-18 04:39:29 - Samuel Totorica (Additional comments)\nHello,\n\nWould it be possible to reset the purging timer for my Cori and Perlmutter scratch folders?  I'm actively using the data in there for a couple of papers simultaneously right now.  In that time I will make sure to learn to use tape storage, but the extension would be a great help becau

In [27]:
data_cleaned.to_csv('/global/u2/m/masare/ticket-text-analysis/bert_ticket_cleaned.csv', index=False)
#data_df = pd.read_csv('/global/u2/m/masare/ticket-text-analysis/Ticket.csv')

In [28]:
#import pandas as pd
#import spacy
#nlp = spacy.load("en_core_web_lg")
#def remove_person_names(text):
  #doc = nlp(text)
 #new_text = []
 # for token in doc:
  #  if token.ent_type_ != "PERSON":
  #    new_text.append(token.text)
   # else:
  #    new_text.append(" ")
 # return " ".join(new_text)



In [29]:
# Apply the function to the relevant columns
#for column in ['Additionalcomments', 'Closenotes', 'CommentsandWorknotes', 'Staffworknotes(NERSCprivate)']:
 # data[column] = data[column].apply(remove_person_names)

#print(data)

In [30]:

  
    # Removing timestamps
    #text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}:\d{2}\s+\w{2}\b', '', text)
    #text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}\d{2}\s+\w{2}\b', '', text, flags=re.IGNORECASE)
    #text = re.sub(r'\b(?:mon|tue|wed|thu|fri|sat|sun)\s+(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\s+\d{1,2}\s+\d{4}\s+\d{1,2}[:]\d{2}\s+\w{2}\b', '', text, flags=re.IGNORECASE)
    #text = re.sub(r'\b\w{3}\s+\w{3}\s+\d{1,2}\s+\d{4}\s+\d{1,2}\d{2}\s+\w{2}\b', '', text) 
    #text = re.sub(r'\b\(?:sat|sun|mon|tue|wed|thu|fri)\.?(?:\w*day)?\s?\b', '',text)
    #mon[a-z\.]*\s+|tue[a-z\.]*\s+|wed[a-z\.]*\s+|thu[a-z\.]*\s+|fri[a-z\.]*\s+|sat[a-z\.]*\s+|sun[a-z\.]*\s+

    
    # Removing bracketed content
    #text = re.sub(r'\[.*?\]', '', text)
    #text = re.sub(r'\(.*?\)', '', text)
    # Removing extra whitespace
    #text = ' '.join(text.split())

    #Removing names
    #text = re.sub(r'\b[a-zA-Z][a-z]*\b(?:\s+[a-zA-Z][a-z]*)*', '', text) 
    #text = re.sub(r'\b[a-zA-Z-]+$\b','', text)
    #phone numbers
    #text = re.sub(r'\d\d\d-\d\d\d-\d\d\d\d','', text)
  
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Additional comments\)', '', text)
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+-\w+ \w+ \(Additional comments\)', '', text)
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - Guest \(Additional comments\)', '', text)
    ##text  = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - System \(Additional comments\)', '', text)
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Additional comments\)', '', text)
    
      #text = re.sub(r'\w+, w+ \d+, d{4} at \d{2}:\d{2} \w{2} \d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)','',text)

    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)', '', text)
    ##text = re.sub(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - Guest \(Staff work notes \(NERSC private\)\)', '', text)
    #text = re.sub(r'On \w+, w+ \d+, d{4} at \d{2}:\d{2} \w{2} \d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2} - \w+ \w+ \(Staff work notes \(NERSC private\)\)'
            
    
    #text = re.sub(r'@\w+ \w+', '', text)
    #text = re.sub(r'\d{4}-\d{2}-\d{2}', '', text)
    # text_data = re.sub(r'\d{2}:\d{2}:\d{2}', '', text) #remove system times
    # text_data = re.sub(r'\(Additional comments\)', '', text)
    # text_data = re.sub(r'\(Staff work notes \(NERSC private\)\)', '', text)
   
    
  
    # Remove punctuations
    # text = re.sub(r'\.','', text)
    #text = re.sub(r',','', text)
    #text = re.sub(r':','', text)
    #text = re.sub(r'\?','', text)
    
   
  
        
    # removing quotes can be problematic
    # text = re.sub(r'\"','', text)
    # text = re.sub(r'\'','', text)
    #text = text.split()
    
    #wl = WordNetLemmatizer()
    #text = [wl.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    #text = ' '.join(text)

In [31]:
#text = "2023-01-30 02:00:04 - System (Additional comments) \
#Incident automatically closed after 7 days in the Resolved state. \
#2023-01-19 07:58:19 - Yuen-Dat Chan (Additional comments) \
#reply from: ydchan@lbl.gov \
#Thank you very much! \
#On Wed, Jan 18, 2023 at 8:12 PM \
#2023-01-18 20:11:28 - Shane Canon (Additional comments) \
#This should be in place now. \
#2023-01-18 13:30:20 - Lisa Gerhardt (Additional comments) \
#Hi Yuen-Dat, \
#I've put in the request to add user chy to the majorana \
#ssh access. You should hear back soon. \
#\
#Bye, \
#Lisa"
